In [1]:
library(tidyverse)

# Custom package
library(rutils)


-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
# Define constants and load data
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

event_code <- list("Alive" = 0, "Dead" = 1)
# covariate_cols_no_figo <- c("age_at_diagnosis", "race", "ethnicity")
# covariate_cols <- c("figo_stage", covariate_cols_no_figo)
dep_cols <- c("vital_status", "survival_time")

In [35]:
dset_idx <- 2

In [36]:
unified_dsets[dset_idx]

[1] "unified_uterine_data"

# Overall

In [37]:
counts_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/counts.tsv")) %>%
    rename(geneID = Hugo_Symbol)
m_counts_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/matrisome_counts.tsv"))


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  Hugo_Symbol = col_character()
)
i Use `spec()` for the full column specifications.



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  geneID = col_character()
)
i Use `spec()` for the full column specifications.




In [38]:
length(counts_df$geneID)

[1] 20242

In [39]:
length(m_counts_df$geneID)

[1] 1008

# Tumor & normal

In [40]:
coldata_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/coldata.tsv"))


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  sample_name = col_character(),
  condition = col_character(),
  data_source = col_character()
)




In [41]:
coldata_df %>% group_by(condition) %>% count()
nrow(coldata_df)

condition,n
<chr>,<int>
healthy,105
tumor,47


[1] 152

# Survival

In [42]:
# Load and filter survival data
survival_path <- paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/survival_data.tsv")
survival_df <- load_survival_df(survival_path, event_code)
filtered_survival_df <- survival_df %>%
    dplyr::select(sample_name, vital_status, survival_time) %>%
    dplyr::filter(vital_status == event_code$Dead, rowSums(is.na(.)) == 0)

# Load normalized matrisome count data
norm_matrisome_counts_path <- paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/norm_matrisome_counts.tsv")

norm_survival_counts_df <- read_tsv(norm_matrisome_counts_path) %>%
    dplyr::select(c("geneID", filtered_survival_df$sample_name))

norm_matrisome_counts_t_df <- read_tsv(norm_matrisome_counts_path) %>%
    column_to_rownames(var = "geneID") %>%
    t() %>%
    as_tibble(rownames = "sample_name")

norm_survival_counts_t_df <- norm_survival_counts_df %>%
    column_to_rownames(var = "geneID") %>%
    t() %>%
    as_tibble(rownames = "sample_name") %>%
    inner_join(filtered_survival_df, by = "sample_name") %>%
    dplyr::select(sample_name, survival_time, everything(), -vital_status)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  sample_name = col_character(),
  vital_status = col_character(),
  survival_time = col_double(),
  figo_stage = col_character(),
  days_to_last_follow_up = col_double(),
  days_to_death = col_double(),
  age_at_diagnosis = col_double(),
  age_at_index = col_double(),
  height = col_double(),
  weight = col_double(),
  bmi = col_double(),
  race = col_character(),
  ethnicity = col_character()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  geneID = col_character()
)
i Use `spec()` for the full column specifications.



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  geneID = col_character()
)
i Use `spec()` for the full column specifications

In [43]:
full_cohort <- nrow(norm_matrisome_counts_t_df)
dec_cohort <- nrow(norm_survival_counts_t_df)

full_cohort
dec_cohort
dec_cohort / full_cohort

[1] 152

[1] 27

[1] 0.1776316